## functions in util module

In [1]:
# merging new's (movieID, title) with old's title so that detailed roles (crew information which old data seem to have??) can be added.
# cinemagoer can be used for teh new's template ia.getmovieID()?

In [3]:
import string
import pandas as pd
from src.match_format import txt2df, tsv2df
from src.join_filter import movie, person, movie_person
from src.check_consistent import print_shape, multi_col_unique, movie_compare
import numpy
from imdb import Cinemagoer
import os
# test data: old_movie = pd.read_pickle('data/movie/old_movie.pkl')

In [53]:
#movie_person()
pmc = pd.read_pickle("new_person_movie_character.pkl")

In [56]:
pmc

,nconst,tconst,category,characters
0,nm0000001,tt0025164,actor,"[""Guy Holden""]"
1,nm0000001,tt0026942,actor,"[""Huck Haines""]"
2,nm0000001,tt0027125,actor,"[""Jerry Travers""]"
3,nm0000001,tt0027630,actor,"[""Bake Baker""]"
4,nm0000001,tt0028333,actor,"[""Lucky Garnett""]"
...,...,...,...,...
52467412,nm9993708,tt14508814,actress,"[""Extra""]"
52467415,nm9993708,tt21028920,actress,\N
52467420,nm9993709,tt11702702,actress,"[""Observant Poet""]"
52467423,nm9993709,tt11772842,actress,"[""Lu""]"


In [54]:
pmc_person = pmc.nconst.unique() # 2m -

In [30]:
pmc_movie = pmc.tconst.unique() #filter only the us-based ones
pmc_movie

array(['tt0025164', 'tt0026942', 'tt0027125', ..., 'tt8743714',
       'tt8744122', 'tt8744160'], dtype=object)

In [38]:
pmc_movie_df = pd.DataFrame(pmc_movie).rename({"0":"pmc_titleId"}, inplace = True)

In [47]:
pmc_movie = pd.DataFrame(pmc_movie)
pmc_movie.columns = ["titleId"] #rename({"0":"pmc_titleId"})
pmc_movie

,titleId
0,tt0025164
1,tt0026942
2,tt0027125
3,tt0027630
4,tt0028333
...,...
5152576,tt8743674
5152577,tt8743704
5152578,tt8743714
5152579,tt8744122


In [60]:
mi = pd.read_csv("mi.tsv", sep = "\t")
mi.rename(columns={"titleId": "tconst"}, inplace=True)
mi

,tconst,old_id,title,year
0,tt0808541,OLD60,Winter,2006
1,tt0970184,OLD218,Operation Homecoming: Writing the Wartime Expe...,2007
2,tt1819776,OLD2120,Virus,2011
3,tt1564349,OLD2185,Dolphin Tale,2011
4,tt2133340,OLD2216,The Foreigner,2011
...,...,...,...,...
93277,tt0097994,OLD2544715,Nobody's Perfect,1990
93278,tt0034166,OLD2544825,Secret of the Wastelands,1941
93279,tt0021078,OLD2544845,Little Accident,1930
93280,tt0023539,OLD2544867,Sunset Trail,1932


In [61]:
df = pd.merge(pmc, mi, on = 'tconst')

In [62]:
df #350k

,nconst,tconst,category,characters,old_id,title,year
0,nm0000001,tt0025164,actor,"[""Guy Holden""]",OLD2390344,The Gay Divorcee,1934
1,nm0001677,tt0025164,actress,"[""Mimi Glossop""]",OLD2390344,The Gay Divorcee,1934
2,nm0002143,tt0025164,actor,"[""Egbert 'Pinky' Fitzgerald""]",OLD2390344,The Gay Divorcee,1934
3,nm0103567,tt0025164,actress,"[""Aunt Hortense""]",OLD2390344,The Gay Divorcee,1934
4,nm0000001,tt0026942,actor,"[""Huck Haines""]",OLD2242602,Roberta,1935
...,...,...,...,...,...,...,...
346239,nm9832080,tt1730714,actor,"[""Topaz""]",OLD2224616,Rubble Kings,2010
346240,nm9888177,tt2181837,actor,\N,OLD1747385,Bending Steel,2013
346241,nm9953110,tt1418781,actor,"[""Guy sleeping in the Fire""]",OLD1691083,A Family Underground,2009
346242,nm9982556,tt8718492,actor,"[""Vision""]",OLD2139024,Miles Away,2011


In [63]:
df_outer = pd.merge(pmc, mi, on = 'tconst', how = 'outer')

In [64]:
df_outer

,nconst,tconst,category,characters,old_id,title,year
0,nm0000001,tt0025164,actor,"[""Guy Holden""]",OLD2390344,The Gay Divorcee,1934.0
1,nm0001677,tt0025164,actress,"[""Mimi Glossop""]",OLD2390344,The Gay Divorcee,1934.0
2,nm0002143,tt0025164,actor,"[""Egbert 'Pinky' Fitzgerald""]",OLD2390344,The Gay Divorcee,1934.0
3,nm0103567,tt0025164,actress,"[""Aunt Hortense""]",OLD2390344,The Gay Divorcee,1934.0
4,nm0000001,tt0026942,actor,"[""Huck Haines""]",OLD2242602,Roberta,1935.0
...,...,...,...,...,...,...,...
20546700,NaN,tt2015531,NaN,NaN,OLD2542534,Writing Down the River,2000.0
20546701,NaN,tt1533113,NaN,NaN,OLD2542535,Where Do You Stand? Stories from an American Mill,2004.0
20546702,NaN,tt0334700,NaN,NaN,OLD2542554,What to Do About Upset Feelings,1964.0
20546703,NaN,tt0194553,NaN,NaN,OLD2542672,World of Comedy,1962.0


In [51]:
# tt2951482, tt3081480 (Breaking Point 2013) both in?b

,titleId,old_id,title,year
0,tt0808541,OLD60,Winter,2006
1,tt0970184,OLD218,Operation Homecoming: Writing the Wartime Expe...,2007
2,tt1819776,OLD2120,Virus,2011
3,tt1564349,OLD2185,Dolphin Tale,2011
4,tt2133340,OLD2216,The Foreigner,2011
...,...,...,...,...
93277,tt0097994,OLD2544715,Nobody's Perfect,1990
93278,tt0034166,OLD2544825,Secret of the Wastelands,1941
93279,tt0021078,OLD2544845,Little Accident,1930
93280,tt0023539,OLD2544867,Sunset Trail,1932


In [65]:
df[df.tconst == "tt2951482"]

,nconst,tconst,category,characters,old_id,title,year
296461,nm1043487,tt2951482,actress,"[""News Reporter""]",OLD12777,Breaking Point,2013
296462,nm2071296,tt2951482,actor,"[""Jimmy Stephens""]",OLD12777,Breaking Point,2013
296463,nm4284309,tt2951482,actress,"[""Jen Stephens""]",OLD12777,Breaking Point,2013
296464,nm4285118,tt2951482,actress,"[""Store Cashier""]",OLD12777,Breaking Point,2013
296465,nm5225546,tt2951482,actor,"[""Jesse Stephens""]",OLD12777,Breaking Point,2013
296466,nm5702978,tt2951482,actress,"[""Store Customer""]",OLD12777,Breaking Point,2013


In [66]:
df[df.tconst == "tt3081480"]

,nconst,tconst,category,characters,old_id,title,year
301223,nm12056369,tt3081480,actress,"[""Girlfriend""]",OLD12777,Breaking Point,2013
301224,nm2415387,tt3081480,actor,"[""Brandon Nichols""]",OLD12777,Breaking Point,2013
301225,nm4390552,tt3081480,actress,"[""Michelle""]",OLD12777,Breaking Point,2013
301226,nm5054020,tt3081480,actor,"[""DJay""]",OLD12777,Breaking Point,2013
301227,nm5521927,tt3081480,actor,"[""Minister""]",OLD12777,Breaking Point,2013
301228,nm5845571,tt3081480,actress,"[""Mary Anne""]",OLD12777,Breaking Point,2013


In [50]:
df

,titleId,old_id,title,year
0,tt0025164,OLD2390344,The Gay Divorcee,1934
1,tt0026942,OLD2242602,Roberta,1935
2,tt0027125,OLD2465834,Top Hat,1935
3,tt0027630,OLD1926061,Follow the Fleet,1936
4,tt0028333,OLD2335959,Swing Time,1936
...,...,...,...,...
79761,tt12239850,OLD2154954,Nightfall,2012
79762,tt1730714,OLD2224616,Rubble Kings,2010
79763,tt2181837,OLD1747385,Bending Steel,2013
79764,tt1418781,OLD1691083,A Family Underground,2009


## Movie titles
- what is in old but not in new
- what is in new but not in old
how to compare strings with string?

In [4]:
movie()

/Users/hyunjimoon/Dropbox/tolzul/BayesSD/ContinuousCode/3_Data4DM/durable_dominance/cont_code/src/join_filter.py:82: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'year'] = df.year.apply(lambda x: int(x) if str(x).isnumeric() else -1)


In [5]:
old_movie = pd.read_pickle('old_movie_year_title.pkl')
new_movie = pd.read_pickle('new_movie_year_title_typemovie.pkl').drop(['region'], axis = 1)

In [30]:
print_shape(old_movie, new_movie)

(1776654, 168973)

In [83]:
ot = multi_col_unique(old_movie)
ot

,old_id,old_id,title,year
0,OLD0,OLD0,Cristián Campos,2010
1,OLD1,OLD1,Life & Death,2013
2,OLD2,OLD2,Disguise for Murder,2001
3,OLD3,OLD3,Anniversary,1971
4,OLD4,OLD4,Angus Black/Lester Barrie/DC Curry,2009
...,...,...,...,...
2545514,OLD2417448,OLD2417448,The Vanquished,1953
2545518,OLD2545518,OLD2545518,V poiskakh kapitana Granta,1985
2545524,OLD1549795,OLD1549795,Winterwatch,2012
2545525,OLD2545525,OLD2545525,The Payoff,1942


In [87]:
old_movie = multi_col_unique(old_movie)
join_movie(old_movie, new_movie)

In [99]:
old_movie

,old_id,title,year
0,OLD0,Cristián Campos,2010
1,OLD1,Life & Death,2013
2,OLD2,Disguise for Murder,2001
3,OLD3,Anniversary,1971
4,OLD4,Angus Black/Lester Barrie/DC Curry,2009
...,...,...,...
2545514,OLD2417448,The Vanquished,1953
2545518,OLD2545518,V poiskakh kapitana Granta,1985
2545524,OLD1549795,Winterwatch,2012
2545525,OLD2545525,The Payoff,1942


In [89]:
mo = pd.read_csv("mo.tsv", sep='\t', low_memory = False) #pd.merge(old_movie, new_movie, on = ('title', 'year'), how = 'inner') # 2m
mi = pd.read_csv("mi.tsv", sep='\t', low_memory = False) #pd.merge(old_movie, new_movie, on = ('title', 'year'), how = 'outer') # 90k

In [90]:
mi #90k

,index,old_id,title,year,titleId
0,0,OLD60,Winter,2006,tt0808541
1,1,OLD218,Operation Homecoming: Writing the Wartime Expe...,2007,tt0970184
2,2,OLD2120,Virus,2011,tt1819776
3,3,OLD2185,Dolphin Tale,2011,tt1564349
4,4,OLD2216,The Foreigner,2011,tt2133340
...,...,...,...,...,...
93277,93277,OLD2544715,Nobody's Perfect,1990,tt0097994
93278,93278,OLD2544825,Secret of the Wastelands,1941,tt0034166
93279,93279,OLD2544845,Little Accident,1930,tt0021078
93280,93280,OLD2544867,Sunset Trail,1932,tt0023539


In [91]:
mo #1.7m

,index,old_id,title,year,titleId
0,0,OLD0,Cristián Campos,2010,NaN
1,1,OLD1,Life & Death,2013,NaN
2,2,OLD2,Disguise for Murder,2001,NaN
3,3,OLD3,Anniversary,1971,NaN
4,4,OLD4,Angus Black/Lester Barrie/DC Curry,2009,NaN
...,...,...,...,...,...
1780098,1780098,NaN,Albatross,2017,tt9914642
1780099,1780099,NaN,9/11: Escape from the Towers,2018,tt9914644
1780100,1780100,NaN,The War of Godzilla,2015,tt9914828
1780101,1780101,NaN,Blind Ambition,2021,tt9914972


In [94]:
old_only = mo[mo.titleId.isnull()] # 1.6m is old\new 1673488

In [95]:
new_only = mo[mo.old_id.isnull()] # 75k is new\old 75544

In [98]:
old_only2 = old_movie[~old_movie.index.isin(new_movie.index)]
new_only2 = new_movie[~new_movie.index.isin(old_movie.index)]
print_shape(old_only2, new_only2)

(1731475, 123794)

In [18]:
old_only

,
title,year
Cristián Campos,2010
Life & Death,2013
Disguise for Murder,2001
Angus Black/Lester Barrie/DC Curry,2009
New Orleans,2003
...,...
Western Spaghetti,2009
V poiskakh kapitana Granta,1985
Winterwatch,2012


In [19]:
new_only

,,titleId,region
title,year,,
"Arrival of Immigrants, Ellis Island",1906,tt0000537,US
The Heart of O Yama,1908,tt0000698,US
The Humpty Dumpty Circus,1898,tt0000704,US
Bill Sharkey's Last Game,1910,tt0000809,US
"Napoleon, the Man of Destiny",1908,tt0000979,US
...,...,...,...
Ancient World Exposed,2019,tt9916348,US
March of Dimes Presents: Once Upon a Dime,1963,tt9916560,US
The Nun 2,2019,tt9916720,US


In [96]:
print(mi.shape[0], "+", old_only.shape[0] , "=", mi.shape[0] + old_only.shape[0], "VS", old_movie.shape[0])
print(mi.shape[0], "+", new_only.shape[0], "=", mi.shape[0] + new_only.shape[0], "VS", new_movie.shape[0])

93282 + 1611538 = 1704820 VS 1776654
93282 + 75283 = 168565 VS 168973


## Person

In [9]:
new_person = person()

In [15]:
old_person

,id,name,imdbIndex,imdbId,gender,namePcodeCf,namePcodeNf,surnamePcode,md5sum
0,451,"A., David",NaN,NaN,m,A313,D13,A,cf45e7b42fbc800c61462988ad1156d2
1,129,"-Alverio, Esteban Rodriguez",NaN,NaN,m,A4162,E2315,A416,f5c410bff6839b545d04c531f776e8f2
2,1688,"Abbas, Athar",NaN,NaN,m,A1236,A3612,A12,cf230f6ed718a330dc688c20b8d741d3
3,3631,"Aberer, Leo",NaN,NaN,m,A164,L16,A16,05684f16b84db387e613d3de57eca5b7
4,512,A.J.,II,NaN,m,A2,NaN,NaN,c63baf59b537e471b6ec4b68c5d1373e
...,...,...,...,...,...,...,...,...,...
4193671,4193674,"Öndemir, Önder",NaN,NaN,NaN,N3565,N3653,N356,4aaaea8e689c667e6f34023f16db5481
4193672,4193676,"Özgür, Lale",NaN,NaN,NaN,Z264,L426,Z26,0b7c397c3605fdc0afbf8220061c1579
4193673,4193670,"Ôta, Akemi",NaN,NaN,NaN,T25,A253,T,ce24ccbc6f95737710b925cd66722d06
4193674,4193675,"Östling, Kia",NaN,NaN,NaN,S3452,K2345,NaN,8c8b06b41b1639845317b12613b0a95b


In [12]:
pmc_raw

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0025164,1,actor,\N,"[""Guy Holden""]"
1,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0026942,2,actor,\N,"[""Huck Haines""]"
2,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0027125,1,actor,\N,"[""Jerry Travers""]"
3,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0027630,1,actor,\N,"[""Bake Baker""]"
4,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0028333,1,actor,\N,"[""Lucky Garnett""]"
...,...,...,...,...,...,...,...,...,...,...,...
52467457,nm9993713,Sambit Mishra,\N,\N,"writer,producer","tt15134202,tt10449366,tt10709066,tt21147342",tt15186906,6,writer,\N,\N
52467458,nm9993713,Sambit Mishra,\N,\N,"writer,producer","tt15134202,tt10449366,tt10709066,tt21147342",tt15187068,6,writer,\N,\N
52467459,nm9993713,Sambit Mishra,\N,\N,"writer,producer","tt15134202,tt10449366,tt10709066,tt21147342",tt15204446,6,writer,\N,\N
52467460,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744,tt8736744,7,cinematographer,\N,\N


In [22]:
pmc.

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
52475552,tt9916880,4,nm10535738,actress,\N,"[""Horrid Henry""]"
52475553,tt9916880,5,nm0996406,director,principal director,\N
52475554,tt9916880,6,nm1482639,writer,\N,\N
52475555,tt9916880,7,nm2586970,writer,books,\N


In [27]:
pmc

,nconst,tconst,category,characters
0,nm0000001,tt0025164,actor,"[""Guy Holden""]"
1,nm0000001,tt0026942,actor,"[""Huck Haines""]"
2,nm0000001,tt0027125,actor,"[""Jerry Travers""]"
3,nm0000001,tt0027630,actor,"[""Bake Baker""]"
4,nm0000001,tt0028333,actor,"[""Lucky Garnett""]"
...,...,...,...,...
52467412,nm9993708,tt14508814,actress,"[""Extra""]"
52467415,nm9993708,tt21028920,actress,\N
52467420,nm9993709,tt11702702,actress,"[""Observant Poet""]"
52467423,nm9993709,tt11772842,actress,"[""Lu""]"


In [104]:


old_person.columns = ['id', 'personID','infoTypeID','info', 'note']

In [109]:
old_person = pd.read_pickle("old_person.pkl")
#new_person = pd.read_pickle('new_person_basics.p6yhn     kl')

In [9]:
new_person_movie = pd.read_pickle('test.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'test.pkl'

In [22]:
os.getcwd()

'/Users/hyunjimoon/Dropbox/tolzul/BayesSD/ContinuousCode/3_Data4DM/empirical_dynamics/cont_code/data/movie'

In [18]:
old_person.columns = ['id', 'name', 'imdbIndex', 'imdbId', 'gender', 'namePcodeCf', 'namePcodeNf', 'surnamePcode', 'md5sum']
old_person[['id', 'name']] #4m

,id,name
0,451,"A., David"
1,129,"-Alverio, Esteban Rodriguez"
2,1688,"Abbas, Athar"
3,3631,"Aberer, Leo"
4,512,A.J.
...,...,...
4193671,4193674,"Öndemir, Önder"
4193672,4193676,"Özgür, Lale"
4193673,4193670,"Ôta, Akemi"
4193674,4193675,"Östling, Kia"


In [10]:
pd.read_csv("cast_info.csv").to_pickle("old_cast_info.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'cast_info.csv'

In [1]:
pd.read_csv("data/movie/new_movie_princ.pkl")

NameError: name 'pd' is not defined

In [19]:
new_person.rename(columns = {'primaryName': 'name', 'birthYear': 'year'}, inplace=True)

In [20]:
new_person

,nconst,name,year,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0057345,tt0056404,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050986,tt0069467,tt0060827"
...,...,...,...,...,...,...
11988018,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt2455546,tt11657662,tt14069590"
11988019,nm9993716,Essias Loberg,\N,\N,NaN,\N
11988020,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
11988021,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


## Cinemagoer

In [ ]:
from imdb import Cinemagoer
ia = Cinemagoer()
object = ia
filehandler = open("Cinemagoer2.pkl", 'w')

import pickle, copyreg, ssl

def save_sslcontext(obj):
    return obj.__class__, (obj.protocol,)

copyreg.pickle(ssl.SSLContext, save_sslcontext)
context = ssl.create_default_context()
foo = pickle.dumps(context, protocol=3)
# _foo = pickle.loads(foo)
# print(_foo)
# #<ssl.SSLContext object at 0x1011812a8>
# print(_foo.protocol)

# when working with engagingdb, mindful of protocol (conda can be of aid)
#pickle.dump(object, filehandler, protocol = 3)

In [ ]:
ia = Cinemagoer()
movie = ia.get_movie('0133093')
for director in movie['directors']:
    print(director['name'])
for genre in movie['genres']:
    print(genre)

In [383]:
people = ia.search_person('Mel Gibson')
for person in people:
   print(person.personID, person['name'])
person = ia.get_person('0000206')
print(person)

0000154 Mel Gibson
12550288 Mel Gibson
2153167 Mel Gibson
0317112 Mel Gibson
3323885 Adam Taylor
2800055 Mel Gibson
3742421 Melina Mari Bryant   nickname
0317195 Terel Gibson
0316945 Donal Gibson
4047961 Noel Gibson
0316878 Cal Gibson
13996602 Joel Gibson
3912400 Jamel Gibson
2106809 Raquel Gibson
0316950 E.L. Gibson
12479102 Gel Gibson
2798174 Daniel Gibson
6928816 Christel Gibson
0316915 Daniel Gibson
1863978 Ethel Gibson
